In [1]:
%reload_ext autotime
import geopandas as gpd
import pandas as pd
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error

In [2]:
# Transects, origin is landward. Has beach_slope
transects = gpd.read_file("transects_extended.geojson")
transects.set_index("id", inplace=True)
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,geometry
id,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nzd0562-0017,nzd0562,NaN,NaN,NaN,0.055,0.0541,0.0608,0.006797,582.0,407.0,0.000019,8.181382,127.722282,11.301428,212.322965,"LINESTRING (174.9093 -36.86951, 174.91331 -36...."
nzd0562-0018,nzd0562,NaN,NaN,NaN,0.045,0.0434,0.0461,0.284894,582.0,372.0,0.022977,9.773159,187.352460,13.687675,216.532589,"LINESTRING (174.90891 -36.86889, 174.91293 -36..."
nzd0563-0000,nzd0563,NaN,NaN,NaN,0.200,0.1756,0.2000,-0.286535,770.0,684.0,0.020982,12.984757,214.309394,14.639310,182.918496,"LINESTRING (176.9374 -39.3345, 176.938 -39.33807)"


In [3]:
vos_files = pd.Series(sorted(glob("shoreline_data_run6/*/time_series_tidally_corrected.csv")))
vos_files = vos_files[~vos_files.str.contains("nzd")]
vos_files

0       shoreline_data_run6/aus0001/time_series_tidall...
1       shoreline_data_run6/aus0002/time_series_tidall...
2       shoreline_data_run6/aus0003/time_series_tidall...
3       shoreline_data_run6/aus0004/time_series_tidall...
4       shoreline_data_run6/aus0005/time_series_tidall...
                              ...                        
3064    shoreline_data_run6/usa_HI_0051/time_series_ti...
3065    shoreline_data_run6/usa_HI_0052/time_series_ti...
3066    shoreline_data_run6/usa_HI_0053/time_series_ti...
3067    shoreline_data_run6/usa_HI_0054/time_series_ti...
3068    shoreline_data_run6/usa_HI_0057/time_series_ti...
Length: 2509, dtype: object

In [4]:
my_nz_files = pd.Series(sorted(glob("data/*/transect_time_series_tidally_corrected.csv")))
my_nz_files

0      data/nzd0001/transect_time_series_tidally_corr...
1      data/nzd0002/transect_time_series_tidally_corr...
2      data/nzd0003/transect_time_series_tidally_corr...
3      data/nzd0004/transect_time_series_tidally_corr...
4      data/nzd0005/transect_time_series_tidally_corr...
                             ...                        
557    data/nzd0559/transect_time_series_tidally_corr...
558    data/nzd0560/transect_time_series_tidally_corr...
559    data/nzd0561/transect_time_series_tidally_corr...
560    data/nzd0562/transect_time_series_tidally_corr...
561    data/nzd0563/transect_time_series_tidally_corr...
Length: 562, dtype: object

In [5]:
files = pd.concat([vos_files, my_nz_files])
files

0      shoreline_data_run6/aus0001/time_series_tidall...
1      shoreline_data_run6/aus0002/time_series_tidall...
2      shoreline_data_run6/aus0003/time_series_tidall...
3      shoreline_data_run6/aus0004/time_series_tidall...
4      shoreline_data_run6/aus0005/time_series_tidall...
                             ...                        
557    data/nzd0559/transect_time_series_tidally_corr...
558    data/nzd0560/transect_time_series_tidally_corr...
559    data/nzd0561/transect_time_series_tidally_corr...
560    data/nzd0562/transect_time_series_tidally_corr...
561    data/nzd0563/transect_time_series_tidally_corr...
Length: 3071, dtype: object

In [6]:
def get_trends(f):
  df = pd.read_csv(f)
  try:
    df.dates = pd.to_datetime(df.dates)
  except:
    print(f)
  df.index = (df.dates - df.dates.min()).dt.days / 365.25
  df.drop(columns=["dates", "satname", "Unnamed: 0"], inplace=True, errors='ignore')
  trends = []
  for transect_id in df.columns:
    sub_df = df[transect_id].dropna()
    if not len(sub_df):
      continue
    x = sub_df.index.to_numpy().reshape(-1, 1)
    y = sub_df
    linear_model = LinearRegression().fit(x, y)
    pred = linear_model.predict(x)
    trends.append({
      "transect_id": transect_id,
      "trend": linear_model.coef_[0],
      "intercept": linear_model.intercept_,
      "n_points": len(df[transect_id]),
      "n_points_nonan": len(sub_df),
      "r2_score": r2_score(y, pred),
      "mae": mean_absolute_error(y, pred),
      "mse": mean_squared_error(y, pred),
      "rmse": root_mean_squared_error(y, pred),
    })
  return pd.DataFrame(trends)
trends = pd.concat(process_map(get_trends, files)).set_index("transect_id")
len(trends)

/tmp/ipykernel_33241/2761047807.py:30: TqdmWarning: Iterable length 3071 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  trends = pd.concat(process_map(get_trends, files)).set_index("transect_id")


  0%|          | 0/3071 [00:00<?, ?it/s]

121933

In [7]:
trends[trends.n_points_nonan > 10].sort_values("r2_score")

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
transect_id,,,,,,,,
nzd0380-0001,0.000003,396.986919,526,434,3.487322e-12,10.354421,180.112991,13.420618
aus0213-0000,-0.000003,211.475107,543,487,8.622880e-12,7.797467,94.136738,9.702409
chi0406-0028,0.000005,183.818769,456,396,1.916567e-11,6.755653,85.309469,9.236312
mex0157-0034,-0.000008,225.225194,496,413,3.072342e-11,10.834152,183.654195,13.551907
usa_CA_0052-0040,-0.000010,226.224974,1340,971,5.089262e-11,10.782984,177.958484,13.340108
...,...,...,...,...,...,...,...,...
mex0004-0007,-7.151559,422.208551,453,432,9.769006e-01,8.743639,126.509611,11.247649
mex0004-0005,-7.365062,424.584864,453,411,9.769376e-01,9.308165,136.176728,11.669479
mex0004-0006,-7.311809,424.851589,453,424,9.771574e-01,9.168770,132.126977,11.494650


In [8]:
trends.describe()

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
count,121933.000000,121933.000000,121933.000000,121933.000000,1.219330e+05,121933.000000,121933.000000,121933.000000
mean,0.234233,238.692519,557.253631,436.155840,1.706041e-01,12.952105,355.154927,16.376453
std,2.027113,97.395974,256.045250,209.935107,2.190084e-01,7.520278,673.613495,9.325633
min,-61.721935,-2765.025713,56.000000,10.000000,3.487322e-12,3.181092,16.314089,4.039070
25%,-0.279281,185.689478,371.000000,282.000000,1.501543e-02,8.693111,122.057271,11.047953
50%,0.128640,217.607824,511.000000,414.000000,7.247818e-02,11.126523,197.550134,14.055253
75%,0.614909,314.021427,672.000000,536.000000,2.420070e-01,14.775802,347.321528,18.636564
max,89.734302,2329.832271,1567.000000,1334.000000,9.788066e-01,172.354249,57339.880098,239.457470


In [9]:
(transects.trend - trends.trend).describe()

count    121955.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: trend, dtype: float64

In [10]:
transects.update(trends.drop_duplicates())

In [11]:
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,geometry
id,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nzd0562-0017,nzd0562,NaN,NaN,NaN,0.055,0.0541,0.0608,0.006797,582.0,407.0,0.000019,8.181382,127.722282,11.301428,212.322965,"LINESTRING (174.9093 -36.86951, 174.91331 -36...."
nzd0562-0018,nzd0562,NaN,NaN,NaN,0.045,0.0434,0.0461,0.284894,582.0,372.0,0.022977,9.773159,187.352460,13.687675,216.532589,"LINESTRING (174.90891 -36.86889, 174.91293 -36..."
nzd0563-0000,nzd0563,NaN,NaN,NaN,0.200,0.1756,0.2000,-0.286535,770.0,684.0,0.020982,12.984757,214.309394,14.639310,182.918496,"LINESTRING (176.9374 -39.3345, 176.938 -39.33807)"


In [12]:
trends.columns, transects.columns, trends.columns.isin(transects.columns)

(Index(['trend', 'intercept', 'n_points', 'n_points_nonan', 'r2_score', 'mae',
        'mse', 'rmse'],
       dtype='object'),
 Index(['site_id', 'orientation', 'along_dist', 'along_dist_norm',
        'beach_slope', 'cil', 'ciu', 'trend', 'n_points', 'n_points_nonan',
        'r2_score', 'mae', 'mse', 'rmse', 'intercept', 'geometry'],
       dtype='object'),
 array([ True,  True,  True,  True,  True,  True,  True,  True]))

In [13]:
transects = transects.join(trends.loc[:,~trends.columns.isin(transects.columns)])
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,geometry
id,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nzd0562-0017,nzd0562,NaN,NaN,NaN,0.055,0.0541,0.0608,0.006797,582.0,407.0,0.000019,8.181382,127.722282,11.301428,212.322965,"LINESTRING (174.9093 -36.86951, 174.91331 -36...."
nzd0562-0018,nzd0562,NaN,NaN,NaN,0.045,0.0434,0.0461,0.284894,582.0,372.0,0.022977,9.773159,187.352460,13.687675,216.532589,"LINESTRING (174.90891 -36.86889, 174.91293 -36..."
nzd0563-0000,nzd0563,NaN,NaN,NaN,0.200,0.1756,0.2000,-0.286535,770.0,684.0,0.020982,12.984757,214.309394,14.639310,182.918496,"LINESTRING (176.9374 -39.3345, 176.938 -39.33807)"


In [14]:
transects.drop_duplicates().to_file("transects_extended.geojson", driver="GeoJSON")